In [1]:
import pycuda.autoinit
import pycuda.gpuarray as gpuarray
import numpy as np
import skcuda.linalg as culinalg

/home/ubuntu/.virtualenvs/pycuda_ve/lib/python3.6/site-packages/skcuda/cublas.py:283: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')


In [2]:
culinalg.init()

In [3]:
np.random.seed(0)

In [4]:
datatype = np.float64

In [5]:
m=50
n=50

In [6]:
a = np.asarray(np.random.rand(m,n), datatype)

In [7]:
print(a)

[[0.5488135  0.71518937 0.60276338 ... 0.1289263  0.31542835 0.36371077]
 [0.57019677 0.43860151 0.98837384 ... 0.02010755 0.82894003 0.00469548]
 [0.67781654 0.27000797 0.73519402 ... 0.91948261 0.7142413  0.99884701]
 ...
 [0.0698143  0.22649128 0.48110196 ... 0.43584121 0.73859222 0.52658426]
 [0.88668322 0.8309088  0.03160544 ... 0.55957053 0.5846446  0.09127072]
 [0.60047104 0.38152221 0.86758085 ... 0.72433098 0.19157101 0.66217423]]


In [8]:
a_gpu = gpuarray.to_gpu(a)

In [9]:
u_gpu, s_gpu, v_gpu = culinalg.svd(a_gpu)

In [10]:
a_rec = np.dot(u_gpu.get(), np.dot(np.diag(s_gpu.get()), v_gpu.get()))

In [11]:
#Error relativo:
np.max(np.abs(a - a_rec)/np.abs(a))

1.915426312164951e-11

In [12]:
import time

In [13]:
m = 1000
n = 1000

In [14]:
np.random.seed(0)

In [15]:
datatype = np.float64

In [16]:
a = np.asarray(np.random.rand(m, n), datatype)

In [17]:
inicio_tiempo = time.time()
a_gpu = gpuarray.to_gpu(a)
fin_load = time.time()

In [18]:
print('tiempo para cargar datos en la GPU:', fin_load - inicio_tiempo)

tiempo para cargar datos en la GPU: 0.002044677734375


In [19]:
gpu_svd_inicio = time.time()
u_gpu, s_gpu, v_gpu = culinalg.svd(a_gpu)
gpu_svd_fin = time.time()

In [20]:
load_datos_inicio = time.time()
u_gpu_result, s_gpu_result, v_gpu_result = u_gpu.get(), s_gpu.get(), v_gpu.get()
load_datos_fin = time.time()

In [21]:
print('tiempo para realizar cargar resultados del device al host:',
      load_datos_fin-load_datos_inicio, 'sec')

tiempo para realizar cargar resultados del device al host: 0.011386871337890625 sec


In [22]:
print('tiempo para realizar la SVD de una matriz en la GPU:',
      gpu_svd_fin-gpu_svd_inicio, 'sec')

tiempo para realizar la SVD de una matriz en la GPU: 4.568828582763672 sec


In [23]:
print('Total (incluyendo load host-device, device-host):', load_datos_fin - inicio_tiempo, 'sec')

Total (incluyendo load host-device, device-host): 4.610806226730347 sec


In [24]:
cpu_svd_inicio = time.time()
u, s, v = np.linalg.svd(a)
cpu_svd_fin = time.time()

In [25]:
print('tiempo para realizar la SVD de una matriz en la GPU:',
      cpu_svd_fin-cpu_svd_inicio, 'sec')

tiempo para realizar la SVD de una matriz en la GPU: 0.569695234298706 sec
